In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import scipy.optimize as spo
import statsmodels.api as sm
from statsmodels.graphics.dotplots import dot_plot
from scipy.interpolate import interp1d
import seaborn as sns
params = {'figure.figsize': (10,7),
          'axes.labelsize': 18,
          'font.size': 18,
          'xtick.labelsize': 16,
          'ytick.labelsize': 16,
          'legend.fontsize': 16}
sns.set(palette='Set2', style='ticks', rc=params)

### Part 1 ###

In [2]:
def pts_to_matrix(points):
    arr = np.zeros((len(points), 2), int)
    for i, (x,y) in enumerate(points):
        arr[i, 0] = x
        arr[i, 1] = y
    return arr

In [3]:
test_data = [(1,1), (1,6), (8,3), (3,4), (5,5), (8,9)]
test_data = pts_to_matrix(test_data)
test_data

array([[1, 1],
       [1, 6],
       [8, 3],
       [3, 4],
       [5, 5],
       [8, 9]])

In [4]:
with open('inputs/day6.txt') as fp:
    real_data = []
    for line in fp.readlines():
        x, _, y = line.partition(',')
        real_data.append((int(x), int(y)))
real_data = pts_to_matrix(real_data)
real_data[0], real_data[-1]

(array([242, 164]), array([236, 191]))

In [5]:
def get_range(data):
    xmin = min(data[:, 0])
    xmax = max(data[:, 0])
    ymin = min(data[:, 1])
    ymax = max(data[:, 1])
    return xmin, xmax, ymin, ymax

In [6]:
get_range(test_data)

(1, 8, 1, 9)

In [7]:
from scipy.spatial.distance import cdist

In [8]:
def closest(pt, data):
    distances = cdist(data, pt, 'cityblock')
    mindist = np.min(distances)
    if list(distances).count(mindist) != 1:
        return None
    return np.argmin(distances)    

In [9]:
assert None == closest(np.array([[5, 0]], int), test_data)
assert 0 == closest(np.array([[0, 0]], int), test_data)

In [10]:
import collections

In [11]:
def build_closest_grid(data):
    xmin, xmax, ymin, ymax = get_range(data)
    grid = {(x,y): closest(np.array([[x,y]], int), data) for x in range(xmin, xmax+1) for y in range(ymin, ymax+1)}
    return (xmin, xmax, ymin, ymax), grid
def find_infinite_indices(ranges, grid):
    xmin, xmax, ymin, ymax = ranges
    bottom_indices = set(grid[(x, ymin)] for x in range(xmin, xmax+1))
    top_indices = set(grid[(x, ymax)] for x in range(xmin, xmax+1))
    left_indices = set(grid[(xmin, y)] for y in range(ymin, ymax+1))
    right_indices = set(grid[(xmax, y)] for y in range(ymin, ymax+1))
    return bottom_indices | top_indices | left_indices | right_indices
def finite_areas(grid, infinite_indices):
    finite_grid = {k:v for k,v in grid.items() if v not in infinite_indices}
    return collections.Counter(finite_grid.values())
def find_biggest_area(data):
    ranges, grid = build_closest_grid(data)
    infinite_indices = find_infinite_indices(ranges, grid)
    areas = finite_areas(grid, infinite_indices)
    return areas.most_common(1)

In [12]:
find_biggest_area(test_data)

[(4, 17)]

In [13]:
find_biggest_area(real_data)

[(47, 5532)]

### Part 2 ###

In [14]:
def total_distance(pt, data):
    return sum(list(cdist(data, pt, 'cityblock')))
def build_total_distance_grid(data):
    xmin, xmax, ymin, ymax = get_range(data)
    grid = {(x,y): total_distance(np.array([[x,y]], int), data)[0] for x in range(xmin, xmax+1)
                                                                for y in range(ymin, ymax+1)}
    return (xmin, xmax, ymin, ymax), grid
def num_close_points(data, limit):
    ranges, grid = build_total_distance_grid(data)
    return len([d for d in grid.values() if d < limit])

In [15]:
assert 16 == num_close_points(test_data, 32)

In [16]:
num_close_points(real_data, 10000)

36216